<a href="https://colab.research.google.com/github/mirza1992/Colab/blob/main/Topic_Modeling_LDA_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import os
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import random
import re
import time
import string
nltk.download('stopwords')
# from nltk.stem import PorterStemmer
# from nltk.stem import LancasterStemmer
nltk.download('punkt')

os.chdir('..')
pd.set_option('display.max_columns', None)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
content = pd.read_pickle('/content/gdrive/My Drive/Colab Notebooks/Dataset/kpmg_full_text_export_2018-2022.pkl')

content['content_full'] = content['headline'] +" "+ content['full_content']

content


# asd = content.groupby('primary_language')['content_full'].apply(' '.join).reset_index()



,bodhi_id,published_at,primary_language,headline,full_content,media_subtype,provider_name,content_full
0,243902440,2020-01-04,en,Family business - the backbone of the economy,Ebrahim Baeshen: Family companies represent mo...,website,None,Family business - the backbone of the economy ...
1,244253478,2020-01-04,en,Family business - the backbone of the economy,Ebrahim Baeshen: Family companies represent mo...,newspaper,None,Family business - the backbone of the economy ...
2,243902505,2020-01-05,en,Responsible business: is this time different?,"This has been the year of profit with purpose,...",newspaper,None,Responsible business: is this time different? ...
3,244749725,2020-01-06,en,Effat student wins EY corporate finance award,"Yara Bugis, a Saudi female student from Effat ...",website,None,Effat student wins EY corporate finance award ...
4,244246407,2020-01-06,en,Yara Bugis Wins EY KSA Corporate Finance Woman...,"Yara Bugis, a Saudi female student from Effat ...",website,None,Yara Bugis Wins EY KSA Corporate Finance Woman...
...,...,...,...,...,...,...,...,...
12594,233227761,2019-12-06,en,Aramco shares to start trading Dec 11: Tadawul...,RIYADH — Saudi Aramco shares will start tradin...,website,None,Aramco shares to start trading Dec 11: Tadawul...
12595,231380652,2019-12-03,en,Deloitte Middle East Point Of View: Disrupt Th...,The oil and gas industry in the Middle East re...,website,None,Deloitte Middle East Point Of View: Disrupt Th...
12596,230634371,2019-12-02,en,Global PwC study ranks Bahrain as the easiest ...,Bahrain has been deemed the easiest place on e...,website,None,Global PwC study ranks Bahrain as the easiest ...
12597,230633415,2019-12-02,en,GCC countries set their sights on knowledge ec...,DUBAI: The member states of the Gulf Cooperati...,website,None,GCC countries set their sights on knowledge ec...


In [5]:
# Check Counts
content.groupby('primary_language', dropna=False).count()


,bodhi_id,published_at,headline,full_content,media_subtype,provider_name,content_full
primary_language,,,,,,,
en,12585,12585,12585,12585,12585,10238,12585
NaN,14,14,14,0,14,0,0


In [6]:
# Filter English (En)
content = content[content['primary_language'] == 'en'].reset_index(drop=True)

content


,bodhi_id,published_at,primary_language,headline,full_content,media_subtype,provider_name,content_full
0,243902440,2020-01-04,en,Family business - the backbone of the economy,Ebrahim Baeshen: Family companies represent mo...,website,None,Family business - the backbone of the economy ...
1,244253478,2020-01-04,en,Family business - the backbone of the economy,Ebrahim Baeshen: Family companies represent mo...,newspaper,None,Family business - the backbone of the economy ...
2,243902505,2020-01-05,en,Responsible business: is this time different?,"This has been the year of profit with purpose,...",newspaper,None,Responsible business: is this time different? ...
3,244749725,2020-01-06,en,Effat student wins EY corporate finance award,"Yara Bugis, a Saudi female student from Effat ...",website,None,Effat student wins EY corporate finance award ...
4,244246407,2020-01-06,en,Yara Bugis Wins EY KSA Corporate Finance Woman...,"Yara Bugis, a Saudi female student from Effat ...",website,None,Yara Bugis Wins EY KSA Corporate Finance Woman...
...,...,...,...,...,...,...,...,...
12580,233227761,2019-12-06,en,Aramco shares to start trading Dec 11: Tadawul...,RIYADH — Saudi Aramco shares will start tradin...,website,None,Aramco shares to start trading Dec 11: Tadawul...
12581,231380652,2019-12-03,en,Deloitte Middle East Point Of View: Disrupt Th...,The oil and gas industry in the Middle East re...,website,None,Deloitte Middle East Point Of View: Disrupt Th...
12582,230634371,2019-12-02,en,Global PwC study ranks Bahrain as the easiest ...,Bahrain has been deemed the easiest place on e...,website,None,Global PwC study ranks Bahrain as the easiest ...
12583,230633415,2019-12-02,en,GCC countries set their sights on knowledge ec...,DUBAI: The member states of the Gulf Cooperati...,website,None,GCC countries set their sights on knowledge ec...


In [7]:
# Pre-Processing

# 1- Detect Language - Arabic

# bodh_id_arr = []

# for fc in range(0,len(content)):
#     print(fc)
#     if len(re.findall(r'[\u0600-\u06FF]+', content['full_content'][fc])) > 0:
#         bodh_id_arr.append(content['bodhi_id'][fc])
#     time.sleep(2)

# print(bodh_id_arr)

content_non_en_list = [341531404, 341804427, 356412760, 361517165, 361514032, 361518459, 361508278, 361508019, 369283058, 373871257, 373870393, 387808114, 387807226, 393995322, 393994693, 393994317, 393991925, 394003279, 393998388, 393998092, 393997833, 393997748, 393996819, 393991548, 393995777, 393993779, 393993535, 397693698, 397694413, 421306492, 421306829, 399634461, 400634292, 403831691, 403831325, 403830942, 403831139, 403836578, 403830737, 404728196, 404728900, 404728553, 404728346, 406012340, 405663558, 406012521, 406012767, 408707253, 408706031, 409126822, 409576542, 416784994, 411511808, 411511643, 412270392, 412269146, 412260802, 412258961, 412265447, 412274289, 412282424, 412280058, 412278941, 412276421, 412278428, 412274887, 412274642, 412275394, 412276104, 413618662, 413616330, 414548976, 414549057, 414548890, 414548760, 414548306, 414548716, 417533376, 417531049, 417530263, 417532000, 417532440, 418772626, 418774112, 418772309, 418774645, 418775389, 418775645, 418774990, 418773921, 419474418, 421692596, 422247068, 423386578, 423386975, 424338244, 424728153, 424728983, 424728545, 425863473, 425613216, 425863596, 426224194, 426223487, 426223855, 426590006, 426589944, 426962811, 426962700, 428102625, 428499532, 428499441, 428499667, 428499945, 428499877, 428498799, 428499063, 428499286, 428499820, 428499738, 428499363, 428869473, 428868971, 428868281, 428869295, 429388853, 429387590, 429390246, 429388464, 429387912, 429389205, 428868569, 428872389, 429390724, 429402958, 429403843, 429389827, 430525022, 430526553, 430526908, 430526664, 430526432, 430528337, 430527514, 430527828, 430527567, 430527376, 430526148, 430528072, 430527901, 430527829, 430528073, 430528779, 430859755, 430860927, 430860889, 430859605, 430860468, 430860680, 430860756, 430861041, 430860297, 430860090, 430859978, 430860382, 431253566, 431255292, 431253662, 431255795, 431255958, 431255099, 431254722, 431254551, 431254272, 431254787, 431254906, 431255015, 431252846, 431253275, 431255191, 431253749, 431253786, 431253903, 431253970, 431254055, 431254152, 431675582, 431676802, 431676767, 431676860, 431676119, 431676548, 431676605, 431676702, 431675295, 431675812, 431675069, 431675150, 431675957, 431675702, 431675755, 432106780, 432107131, 432107726, 432107189, 432107678, 432107512, 432107334, 432107609, 432107553, 432108578, 432107280, 432107058, 432107003, 432107448, 432106916, 432107363, 433156754, 433157811, 433157634, 433157703, 433485299, 433158407, 433156930, 433158736, 433157922, 433158585, 433159063, 433158199, 433158276, 433156983, 433156843, 433157478, 433158083, 433157148, 433482145, 433485106, 433484795, 433158002, 433483664, 433483448, 433482323, 433482434, 433482643, 433482827, 433483908, 433484067, 433484946, 433484672, 433484414, 433484231, 433485023, 433484847, 433899161, 433896532, 433891513, 433900795, 433893128, 433901177, 433894697, 433895324, 433895815, 433897087, 433897966, 433897393, 433899858, 433900423, 433892311, 433893453, 433893934, 433894365, 434348740, 434349838, 434346742, 434347779, 434350179, 434350479, 434350791, 434351082, 434354028, 434346097, 434345634, 434347358, 434345211, 434344867, 434344332, 434353254, 434353784, 434797261, 434796696, 434344001, 434352723, 434354607, 434351355, 434351839, 434352938, 434352431, 434354283, 434354815, 434796975, 434796032, 434795129, 434799847, 434803568, 434803214, 434797769, 434795612, 434794533, 434799210, 434798864, 434798310, 434800358, 434801070, 434802811, 434803009, 434802027, 434802602, 434801449, 435698397, 435698516, 435700143, 435706145, 435699299, 435706643, 435705744, 435706385, 435705629, 435700708, 435702989, 435705017, 435701715, 435704220, 435704754, 435705284, 435702528, 435704547, 435701251, 435968563, 435968623, 435968689, 435968741, 435966889, 435968065, 435967686, 435968231, 435968296, 435968182, 435968388, 435967825, 435968471, 435967248, 435967056, 435968344, 435966990, 435967478, 436349724, 436349796, 436349282, 436349913, 436350280, 436348811, 436350022, 436349527, 436349676, 436350978, 436350112, 436350198, 436350466, 436350374, 436829885, 436830397, 436833771, 436826311, 436824932, 436825467, 436827141, 436831322, 436826966, 436832106, 436832941, 436829466, 436826493, 436830744, 436830138, 436831028, 436833350, 436825967, 436825704, 436828019, 436827625, 436828281, 436828504, 436828932, 437271950, 437272695, 437273352, 437273203, 437272033, 437272554, 437273768, 437273882, 437273479, 437272198, 437273693, 437273628, 437273171, 437273043, 437272319, 437272284, 437272377, 437272448, 437272951, 438216841, 437296150, 438216209, 438218193, 438220388, 438218975, 438216987, 438219706, 438219591, 438219458, 438219964, 438218501, 438217097, 438217359, 438218078, 438218145, 438217937, 438218611, 438219230, 438218315, 438218827, 438450386, 438452678, 438450857, 438450717, 438450236, 438451877, 438451702, 438452026, 438450623, 438452159, 438452078, 438452762, 438452289, 438452207, 438452404, 438452539, 438451073, 438451274, 438896933, 438894493, 438894363, 438896416, 438896750, 438895676, 438896832, 438897142, 438896125, 438896202, 438894595, 438894778, 438895089, 438897018, 438895566, 438895506, 438895426, 438895342, 438895738, 438895786, 438895879, 438895963, 438896009, 438896265, 439312364, 439311509, 439310863, 439310181, 439308015, 439313377, 439307085, 439306832, 439306725, 439313935, 439307944, 439306062, 439313226, 439308122, 439314738, 439314211, 439314556, 439314944, 439307838, 439307518, 439307711, 439307606, 439306617, 439306442, 439307248, 439306246, 439306373, 439831712, 439831876, 439828341, 439830861, 439831437, 439831621, 439832001, 439832237, 439831226, 439828703, 439828847, 439828484, 439830088, 439829636, 439829388, 439829142, 439829002, 441107445, 441106501, 441106351, 441105490, 441107597, 441107747, 441107927, 441108583, 441108875, 441107245, 441106639, 441107516, 441108055, 441108320, 441108433, 441106292, 441106040, 441105827, 441441462, 441433882, 441434514, 441434146, 441432975, 441435235, 441433588, 441432312, 441431893, 441431575, 441431238, 441444904, 441448838, 441442185, 441448543, 441448109, 441447773, 441445417, 441447115, 441954948, 442434412, 441441752, 441432635, 441432743, 441430713, 441430198, 441433384, 441446470, 441445080, 441444533, 441444154, 441443780, 441443257, 441449172, 441442479, 441442720, 441442897, 441447383, 441445755, 441446274, 441455127, 441937080, 441945771, 441947902, 441937279, 441937371, 441937907, 441938152, 441938990, 441938640, 441942687, 441942007, 441939376, 441939816, 441944863, 441956460, 441945326, 441937542, 442933467, 441940271, 441944230, 441947145, 441946704, 441946946, 441947361, 441946107, 441946465, 441944622, 441947654, 441943811, 441943538, 441943196, 441956930, 441955664, 442435448, 444133001, 442436925, 442435887, 442437160, 442436523, 442436310, 442437641, 442437293, 442437889, 442434243, 442937462, 442436607, 442437429, 442434823, 442434777, 442438316, 442436148, 442435754, 442435118, 442434687, 442934268, 442933575, 442937726, 442935165, 442935279, 442934924, 442934350, 442934482, 442934653, 442933987, 442934137, 442935360, 442933668, 442933737, 442933794, 442933884, 442934059, 444128791, 444129781, 444130110, 444129090, 444129677, 444130276, 444128589, 444130399, 444128496, 444127941, 444129354, 444129298, 444129578, 444130027, 444128316, 444128872, 444129046, 444129886, 444130221, 444128971, 444128414, 444128222, 444128054, 444130503, 444469679, 444469484, 444467449, 444468679, 444129940, 444469854, 444469972, 444470021, 444470101, 444470178, 444470401, 444470491, 444470597, 444470703, 444475673, 444475490, 444469385, 444469206, 444469022, 444468917, 444468787, 444468603, 444468321, 444963185, 444963062, 444962969, 444962768, 444962001, 444963786, 444963937, 444964174, 444964307, 444964396, 444963272, 444964468, 444964565, 444964638, 444963632, 444963470, 445479121, 445480178, 445475227, 445479008, 445476881, 445477512, 445479262, 445472006, 445472250, 445472509, 445472760, 445470539, 445474196, 445469221, 445474368, 445469709, 445473864, 445470348, 445471760, 445470961, 445471200, 445471507, 445475582, 445979758, 445977340, 445986085, 445991072, 445470141, 445978255, 445478142, 445478822, 445474981, 445476076, 445478418, 445475921, 445475738, 445475463, 445478560, 445477061, 445476340, 445476640, 445476666, 445477931, 445477311, 445479497, 445982630, 445980646, 445980445, 445982101, 445978625, 447058679, 445980837, 445982772, 445982396, 445981644, 445981077, 445981788, 445981859, 447060554, 447059787, 447060236, 447059130, 447065343, 447059261, 447059415, 447060596, 447060143, 447061523, 447061162, 447061594, 447065907, 447058934, 447061385, 447061428, 447060994, 447061429, 447060787, 447061042, 447061490, 447061488, 447061281, 447061484, 447061116, 447341913, 447345008, 447348197, 447347596, 447345612, 447344836, 447343993, 447346209, 447345791, 447342995, 447346032, 447343672, 447343812, 447344311, 447865440, 447864546, 447868141, 447864953, 447866967, 447867167, 447867671, 447867294, 447867455, 447867899, 448381954, 448376680, 448380745, 448381256, 448379675, 448381579, 448387963, 448387749, 448387523, 448378648, 447864004, 448379970, 448380226, 448380406, 448377026, 448377294, 448377476, 448377671, 448378000, 448379299, 448379009, 448378450, 448378237, 448897752, 448897459, 448896942, 448899428, 448899545, 448896224, 455180176, 455169365, 448900090, 448898746, 448898846, 448898998, 448899169, 448898155, 448898012, 448899295, 448896688, 448896474, 448898654, 448898320, 450284146, 450288515, 450287399, 450287174, 450287975, 450641733, 450286419, 450285736, 450288124, 450284271, 450288178, 450287526, 450287561, 450287477, 450287786, 450287904, 450287011, 450286798, 450286573, 450288265, 450288647, 450640835, 450636974, 450638996, 450639852, 450642677, 450635052, 450634660, 450634354, 450634061, 451786897, 450635761, 450637768, 450635956, 450636222, 450633120, 450633476, 450632809, 450635597, 450635394, 451209517, 451202258, 451203198, 451206205, 451203466, 451209768, 451206740, 451210373, 451201995, 451202761, 451205089, 451205229, 451205466, 451205900, 451205700, 451203686, 451204725, 451785610, 451786479, 451783486, 451786087, 451779684, 451784954, 451784352, 451783817, 451781659, 451782448, 451780435, 451778917, 451777667, 451778139, 451779189, 451782722, 451782999, 451781165, 451782168, 452384162, 452383626, 452381335, 452404795, 452402507, 452402405, 452401301, 452399905, 452399174, 452380250, 452383310, 452380666, 452382903, 452381853, 452400227, 452402880, 452400594, 452401888, 452400476, 452400880, 452403047, 452402978, 452402236, 452401001, 453716856, 453719190, 453717488, 453717644, 453716538, 453717299, 453716366, 453716040, 453715432, 453717595, 453717001, 453717055, 453717126, 453715728, 453716689, 453716589, 453715842, 453715507, 453715577, 453715209, 453717212, 454066371, 454062365, 454064567, 454064897, 454065768, 454065911, 454066019, 454606841, 454065173, 454065461, 454065649, 454064168, 454064087, 454066206, 454063729, 454062042, 454063370, 454063127, 454064320, 454064052, 454063946, 454063858, 454062632, 454606534, 454606356, 454594297, 454608429, 454603103, 454596114, 454605996, 454597695, 454595122, 454595666, 454602816, 454596555, 454596993, 454604898, 454600334, 454605174, 455183020, 455168414, 455158962, 455164819, 455164889, 455159979, 455180819, 455167709, 455171451, 455167224, 455169137, 455166333, 455171994, 455168186, 455166419, 455166352, 455174488, 455183811, 455166337, 455166341, 455167997, 455175551, 455182474, 455184681, 455167530, 455166354, 455183632, 455173700, 455165816, 456996880, 455184201, 455165665, 455166328, 455182608, 455170310, 455625193, 455169777, 455164643, 455162326, 455161633, 455163263, 455162884, 455162188, 455162596, 455161308, 455160554, 455161006, 455159505, 455162044, 455629325, 455627690, 455624592, 455649592, 455643984, 455643303, 455645319, 455644350, 455644790, 455645142, 455644444, 455643659, 455643024, 455642843, 455642319, 455641445, 457001355, 457001753, 456995255, 456994589, 456996339, 457001473, 457354757, 456995966, 456995899, 456995552, 456995040, 456995791, 456996161, 457001265, 457354459, 457353553, 457352990, 457351531, 457351101, 457350600, 457352464, 457351796, 457353987, 457353299, 457352218, 457348037, 457350214, 457349613, 457348578, 457348950, 457349390, 457751343, 457751099, 457750769, 457750543, 457749547, 457751633, 457750343, 457754917, 457749068, 457747619, 457748083, 457749300, 457753776, 457753637, 457753942, 457753507, 457753339, 457753260, 457752907, 457752828, 457752547, 457752091, 458107507, 458111030, 458110688, 458110278, 458110086, 458109860, 458105767, 458105440, 458115310, 458108575, 458109406, 458109617, 458108956, 458107778, 458106829, 458106415, 458106037, 458470274, 458470153, 458469009, 458469628, 458471603, 458469728, 458469962, 458469497, 458469171, 458469062, 458469864, 458470055, 459431078, 459431736, 459431007, 459431979, 459431576, 459432855, 459431164, 459431347, 459432193, 459431468, 459432282, 459431695, 459432321, 459431796, 459432679, 459431836, 459690852, 459690672, 459690992, 459700052, 459697626, 459696009, 459694210, 460089150, 460088602, 459698298, 459699756, 459696816, 459696956, 459696519, 459696367, 459696108, 459695811, 459694897, 459694540, 459694459, 459695602, 459695191, 460090746, 460091194, 460092109, 460092408, 460090287, 460091943, 460091450, 460090088, 460486549, 460485317, 460485035, 460482574, 460479244, 460492430, 460491944, 460485826, 460485529, 460484425, 460484241, 460482047, 460483873, 460483534, 460481181, 460479882, 460480814, 460480421, 460868215, 460867801, 460869419, 460869655, 460879077, 460869134, 460869960, 460868841, 460868628, 460867426, 460866192, 460870341, 460878765, 460878986, 461777139, 461776036, 461777769, 461777615, 461776128, 461777874, 461776203, 461777529, 461776516, 461776625, 461777447, 461777962, 461777214, 461776401, 461776301, 461776485, 462044347, 462044643, 462061628, 462061898, 462065003, 462061415, 462062369, 462063753, 462064130, 462064659, 462064803, 462068013, 462069487, 463217329, 463217489, 462063507, 462044084, 462043468, 462043749, 462043116, 462042826, 462042640, 462042396, 462042342, 462042226, 462042020, 462043630, 462064456, 462424161, 462425517, 462427398, 462428213, 462428418, 462428599, 462438558, 462438082, 462827140, 462426128, 462425881, 462427762, 462425157, 462427121, 462426732, 462424744, 462424624, 462424352, 462426898, 462424891, 462426402, 462829149, 462823709, 462825263, 462825419, 462828557, 463218270, 462823401, 462822859, 462822677, 462824609, 462824973, 462825700, 462825940, 462824874, 462826653, 462826027, 463200126, 463200067, 463200234, 463199825, 463200172, 463199010, 463201002, 463200754, 463218608, 463199292, 463197621, 463200024, 463199956, 463199169, 463200534, 463198314, 463197687, 463198029, 463219966, 464108524, 464114133, 464113786, 464110337, 464109099, 464113177, 464113529, 464111978, 464108908, 464111822, 464112416, 464112653, 464112848, 464109361, 464109684, 464110032, 464110938, 464111240, 464111448, 464352215, 464351968, 464351842, 464353065, 464353214, 464353375, 464351453, 464353471, 464354368, 464354838, 464354970, 464355227, 464356430, 464356750, 464356874, 464355574, 464355707, 464355755, 464356221, 464352868, 464353958, 464353627, 464354619, 464356673, 464355913, 464356275, 464356058, 464350714, 464356486, 464737065, 464736559, 464738045, 464733287, 464732803, 464741493, 464741280, 464740590, 465119870, 465120091, 464735778, 464736218, 464737676, 464738276, 464734777, 464739043, 464738653, 464734492, 464733032, 464734379, 464733657, 464733568, 465106305, 465108056, 465105042, 465104595, 465107671, 465119102, 465106063, 465105599, 465106671, 465101857, 465102016, 465107479, 465103774, 465102532, 465106899, 465107209, 465108521, 465105384, 465104208, 465103612, 465108342, 465108693, 465103036, 465102725, 465105820, 465496926, 465496655, 465494579, 465494459, 465494690, 465494300, 465495244, 465494374, 465496403, 465496105, 465495771, 465496230, 465495548, 465494848, 465495407, 465495092, 465494192, 466372307, 466371585, 466371642, 466372098, 466372444, 466370883, 466371269, 466371787, 466371920, 466372009, 466370317, 466369593, 466369892, 466370566, 466371689, 466686136, 466686358, 466688968, 466685201, 466687536, 466688600, 466688425, 466687204, 466686980, 466688265, 466687975, 466687785, 466685807, 466685639, 466685314, 469271704, 469270360, 469270762, 469267583, 469267766, 469264795, 469271955, 469269856, 469272135, 469269638, 469269470, 469268903, 469619389, 469271093, 469271241, 469268111, 469268505, 469265181, 469268611, 469267187, 469617581, 469618009, 469618315, 469616801, 469618698, 469619107, 469616371, 469618472, 469617081, 469616049, 469615717, 469615375, 469615126, 469614580, 469982445, 469982230, 469981870, 469980160, 469979557, 469979011, 469977454, 469977808, 469982020, 469981649, 469978146, 469981576, 469981192, 469980963, 469980729, 469980531, 469978531, 469978742, 469981453, 470894552, 470896137, 470896325, 470896456, 470894817, 470893809, 470895919, 470893752, 470894185, 470894316, 470894341, 470895980, 470896038, 470896550, 470894407, 471157899, 471157608, 471159372, 471159478, 471159598, 471159662, 471159208, 471158109, 471159149, 471159078, 471159033, 471159002, 471156678, 471157059, 471157406, 471157251, 471517237, 471517411, 471514436, 471514623, 471517538, 471514828, 471517745, 471516471, 471512915, 471513164, 471516070, 471512781, 471912084, 471911908, 471911510, 471912321, 471517023, 471516627, 471515003, 471516754, 471513945, 471517305, 471514063, 471517864, 471515152, 471515221, 471515598, 471515811, 471516302, 471913274, 471913101, 471914414, 471914083, 471912469, 471909495, 471912909, 471912672, 471913564, 471913431, 471911165, 471910782, 471910477, 471910059, 471909818, 472317119, 472318342, 472319068, 472316960, 472316665, 472330086, 472327478, 472328032, 472326254, 472326700, 472327039, 472329548, 472329306, 472329184, 472329052, 472329701, 472329921, 472328862, 472328236, 472328666, 472327359, 473229536, 473229338, 473506331, 473229651, 473506542, 473229812, 473230528, 473230664, 473231623, 473229990, 473230215, 473230326, 473230341, 473230968, 473231343, 473231454, 473231715, 473231846, 473509183, 473509646, 473509794, 473508542, 473508325, 473508424, 473509023, 473508158, 473507994, 473507880, 473507654, 473508906, 473508648, 473903290, 473901228, 473904678, 473903674, 473904109, 473902787, 473902344, 473902253, 473902069, 473903439, 473903126, 473902675, 473900216, 473900007, 473899561, 473903021, 473901854, 473901716, 473900696, 474365807, 474365107, 474365385, 474363948, 474366487, 474366826, 474361470, 474361191, 474360540, 474359644, 474361781, 474367170, 474369834, 474369567, 474369341, 474368507, 474369088, 474364930, 474365004, 474366046, 474364864, 474366164, 474364617, 474364365, 474366283, 474365640, 474367673, 474367835, 474763125, 474757936, 474758400, 474758243, 474769379, 474767996, 474768150, 474769548, 474769719, 474769765, 475628295, 475628139, 475628080, 474769056, 474768329, 474768599, 474769888, 474768784, 474769131, 474769201, 474769265, 474770091, 474770361, 474770813, 475632899, 475879990, 475635870, 475635740, 475635626, 475635450, 475879728, 475634530, 475634464, 475634402, 475634274, 475634084, 475635420, 475635035, 475634014, 475634697, 475634856, 475635214, 475635149, 475635306, 475635376, 475634775, 475634559, 475882750, 475886957, 475887262, 475887109, 475887498, 475883193, 475883337, 475883452, 475883590, 475883781, 475883903, 475884006, 475884157, 475884441, 475884983, 475885235, 475885553, 475885883, 475886016, 475886217, 475886545, 475886682, 475886748, 475886824, 475887314, 475886367, 475882857, 476275383, 476276832, 476278894, 476278309, 476284339, 476289543, 476285154, 476279397, 476278528, 476278088, 476277664, 476277467, 476277348, 476277135, 476276298, 476276462, 476275994, 476276534, 476275558, 476275713, 476275851, 476676017, 476678193, 476677898, 476677525, 476677240, 476677101, 476676862, 476674159, 476678396, 476675465, 476675162, 476676630, 476674927, 476674656, 476674356, 477084865, 477085060, 477084703, 477085414, 477082504, 477086864, 477087057, 477087303, 477083616, 477084474, 477082725, 477083021, 477083390, 477083748, 477085836, 477085552, 477085716, 477083842, 477083974, 477084081, 477084245, 477084346, 477084772, 477084619, 478021561, 478021663, 478021792, 478021882, 478028494, 478027317, 478028625, 478029862, 478027945, 478029365, 478029289, 478029202, 478028955, 478029175, 478029077, 478029054, 478028779, 478028064, 478028013, 478029264, 478029493, 478029523, 478274927, 478274731, 478274623, 478274417, 478274305, 478275047, 478275159, 478275225, 478275440, 478275578, 478280836, 478281084, 478274536, 478275689, 478282600, 478282705, 478282477, 478282412, 478282356, 478282785, 478282875, 478282966, 478283002, 478283082, 478283153, 478282279, 478281239, 478281276, 478281372, 478281920, 478282041, 478282125, 478282213, 478612783, 478624410, 478624543, 478624889, 478625157, 478625818, 478626233, 478628446, 478628260, 478628647, 478625465, 478625551, 478626006, 478624058, 478626278, 478996970, 478998022, 478998267, 479003989, 479003364, 479003668, 479004468, 479005064, 479004759, 479005176, 479005888, 479005566, 479006480, 479002100, 479002693, 479006163, 479002862, 479002256, 479002425, 479002505, 479004341, 479004595, 479494506, 479494002, 479499205, 479499347, 479499509, 479499582, 479499638, 479499708, 479494703, 479496673, 479496186, 479495994, 479496338, 479496904, 479496781, 479498283, 479498889, 479499713, 479498668, 479498552, 479498423, 479498749, 479498995, 479499535, 479499737, 480624166, 480623975, 480625188, 480625522, 480626234, 480624619, 480625931, 480625775, 480626021, 480626112, 480626193, 480626206, 480626220, 480624503, 480624795, 480624937, 480624404, 480977974, 480978615, 480978890, 480975996, 480978383, 480979002, 480979023, 480979303, 480976706, 480976904, 480977037, 480977236, 480976235, 480976345, 480977428, 480979209, 480979044, 480979107, 480979391, 480979456, 480977584, 481414772, 481414420, 481415106, 481416211, 481416533, 481417469, 481418051, 481415661, 481415904, 481416837, 481417121, 481417772, 481834400, 481829523, 481831310, 481831539, 481830514, 481829824, 481832056, 481834756, 481829678, 481830906, 481830630, 481834104, 481830334, 481830397, 481830476, 481831064, 481829927, 481830263, 481830155, 481831764, 481831873, 481831941, 481831744, 482196768, 482197409, 482198124, 482198779, 482199059, 482198614, 482198348, 482203612, 482203041, 482203284, 482196974, 482196499, 482197685, 482197329, 482198533, 482197252, 482197564, 482197812, 482197882, 482198033, 482198187, 482198442, 482199166, 482197915, 483123021, 483122103, 483123182, 483123352, 483121970, 483122255, 483122304, 483122366, 483122506, 483122635, 483122741, 483122829, 483123265, 483122929, 483123508, 483406539, 483407618, 483407811, 483405942, 483408235, 483408373, 483408494, 483745181, 483407737, 483407557, 483406061, 483407886, 483407952, 483408691, 483409075, 483408937, 483408887, 483409659, 483409513, 483409321, 483409143, 483745641, 483743139, 483741128, 483741454, 483743445, 483743475, 483743492, 483744897, 483745321, 483745399, 483740467, 483742059, 483742265, 483742416, 483742915, 483743562, 483744437, 483744660, 483743685, 483744054, 483743847, 483744372, 484101292, 484104063, 484103024, 484105640, 484105036, 484105949, 484457621, 484101482, 484100952, 484100339, 484100091, 484102314, 484101849, 484101706, 484103374, 484103849, 484104378, 484458068, 484457959, 484457873, 484456717, 484456340, 484457746, 484455937, 484459841, 484460123, 484461163, 484457150, 484456987, 484456415, 484456267, 484456123, 484456045, 484457505, 484457356, 484457215, 485310491, 485311710, 485312692, 485312611, 485312067, 485311996, 485312897, 485312843, 485312544, 485312462, 485312404, 485312360, 485312300, 485312217, 485312143, 485312255, 485311878, 485311825, 485311599, 485311463, 485311104, 485310914, 485468768, 485468898, 485466946, 485466097, 485465837, 485466648, 485465725, 485466886, 485466791, 485466732, 485466406, 485466297, 485465924, 485465630, 485465654, 485465750, 485800319, 485799484, 485798310, 485797903, 485798123, 485799307, 485803951, 485798987, 485809723, 485809343, 485809083, 485808810, 485799698, 485799810, 485798707, 485801095, 485801804, 485798503, 485802229, 485798783, 485802869, 485798615, 486158391, 486155943, 486158302, 486157018, 486157319, 486157836, 486163819, 486163555, 486157770, 486155780, 486158152, 486157124, 486157783, 486158038, 486156380, 486156632, 486158232, 486157550, 486158544, 486157951, 486534195, 486532958, 486533349, 486533201, 486539716, 486534020, 486533878, 486539603, 486541034, 486540334, 486540119, 486532297, 486532402, 486532810, 486533657, 486532467, 486532506, 486532561, 486532631, 486533544, 487478617, 487477821, 487478000, 487478148, 487479189, 487477133, 487477081, 487477163, 487477193, 487477407, 487477524, 487477665, 487478434, 487478776, 487478830, 487478909, 487479015, 487479357, 487479118, 487720017, 487719949, 487717513, 487718429, 487717012, 487719409, 487720156, 487718722, 487717981, 487717797, 487717635, 487717335, 487718217, 487720090, 487718058, 487718819, 487719847, 487719235, 487719117, 487718969, 488066065, 488064840, 488065192, 488065439, 488062251, 488067061, 488065807, 488064167, 488064268, 488064421, 488064375, 488064550, 488063506, 488063641, 488063910, 488064105, 488063407, 488063281, 488063461, 488062137, 488062801, 488062936, 488063006, 488064020, 488433900, 488436407, 488436497, 488436918, 488436669, 488437155, 488434629, 488434063, 488435752, 488450705, 488449070, 488449757, 488450012, 488435048, 488433384, 488433468, 488433483, 488437300, 488433652, 488433751, 488436259, 488436091, 488435982, 488839820, 488840730, 488840114, 488837996, 488838054, 488838422, 488839984, 488839174, 488839604, 488842713, 488838923, 488839675, 488839458, 488838818, 488838138, 488838246, 488838301, 488838690, 488839525, 491867212, 491871297, 491867441, 491868045, 491871042, 491870931, 491870682, 491871165, 491868954, 491868000, 491868245, 491868163, 491870804, 491871400, 492130618, 492130698, 491868544, 491867921, 491868389, 491868710, 491868466, 491868458, 491867848, 491867775, 491867736, 491867502, 491871695, 492118699, 492115813, 492117303, 492128630, 492127276, 492126489, 492126056, 492125843, 492128330, 492115943, 492118223, 492116149, 492116403, 492116599, 492117190, 492117572, 492117800, 492118451, 492126754, 492499484, 492499208, 492506590, 492500875, 492499708, 492500742, 492499648, 492499902, 492500003, 492501248, 492500142, 492500938, 492500300, 492500617, 492500524, 492500337, 492885087, 492887424, 492887140, 492884750, 492884593, 492886989, 492886283, 492894849, 492895185, 492895510, 492895743, 492896179, 492896263, 492896296, 492894362, 494466752, 492887235, 492885260, 492885524, 492886867, 492887621, 492886721, 492887837, 492886557, 492885876, 492885970, 492886473, 492886115, 492885996, 493316339, 493311686, 493311183, 493315790, 493316894, 493313720, 493314067, 493314375, 494466556, 493309913, 493309388, 493310294, 493310437, 493310712, 493315561, 493312320, 493312046, 493312809, 493313048, 493313606, 493317122, 493317685, 493314219, 494467090, 494466945, 494469994, 494467781, 494472308, 494471506, 494469685, 494471066, 494471328, 494471675, 494472363, 494471932, 494469775, 494470836, 494470746, 494470629, 494470579, 494468855, 494470386, 494468492, 494469442, 494469587, 494470210, 494470050, 494471797, 494471999, 494472099, 494472210, 494472162, 494807723, 494807518, 494808262, 494808804, 494809126, 494809243, 494809332, 494807381, 494807307, 495222254, 495221536, 495220491, 495223421, 495223742, 495223867, 495223536, 495223635, 495223278, 495224022, 495221968, 495221759, 495221685, 495221247, 495221054, 495220121, 495220281, 495220815, 495637592, 495641998, 495638022, 495642688, 495639513, 495639736, 495642767, 495637816, 495641802, 495638333, 495640900, 495642538, 495642308, 495642259, 495638633, 495641345, 495641136, 495639017, 495639109, 495639306, 495640964, 495640721, 495640440, 496008423, 496004389, 496002716, 496004097, 496004203, 496003926, 496003669, 496003529, 496003578, 496003492, 496003458, 496001880, 496002032, 496001675, 496001628, 496003189, 496002913, 496002757, 496002302, 496002458, 496001549, 496008274, 496992080, 496992145, 496991224, 496990912, 496992007, 496991960, 496991413, 496990719, 496990636, 496990501, 496990466, 496990349, 496990334, 496990125, 496990066, 496991684, 496992226, 496991716, 496991848, 496991810, 497255637, 497250583, 497253073, 497253158, 497256161, 497253248, 497249056, 497248707, 497251307, 497250200, 497251248, 497251019, 497250891, 497250300, 497251421, 497251505, 497250117, 497249448, 497253427, 497251164, 497254647, 497253683, 497253811, 497630349, 497627740, 497629538, 497625570, 497627393, 497658706, 497658931, 497630154, 497626421, 497626097, 497629865, 497628913, 497627204, 497627047, 497626845, 497626725, 497626642, 497625174, 497628861, 497628766, 497628662, 497628519, 497628360, 497628256, 497630078, 497630069, 497629963, 497658348, 498032169, 498030147, 498029948, 498028786, 498032500, 498035885, 500010413, 500010160, 498028359, 498035680, 498035456, 498027869, 498035314, 498035854, 498029821, 498031632, 498031934, 498028486, 498032960, 498029095, 498029619, 498029020, 498028926, 498031152, 498030933, 498030612, 498030363, 498031395, 498032821, 498030760, 500018138, 500017736, 500017654, 500017975, 500016183, 500015596, 500014796, 500018368, 500017270, 500017085, 500017519, 500016901, 500016673, 500015936, 500014043, 500013668, 500013491, 500013047, 500012740, 500012499, 500012184, 500011694, 500039501, 500040631, 500040648, 500039964, 500040151, 500040294, 500040744, 500040835, 500039150, 500039342, 500041130, 500041335, 500041460, 500041561, 500041203, 500531825, 500530116, 500529509, 500529290, 500528915, 500532737, 500532959, 500528672, 500532045, 500532500, 500534188, 500533035, 500533893, 500532245, 501205504, 501204853, 501200852, 501202449, 501204305, 501204040, 501202000, 501200227, 501203296, 501203737, 501203582, 501200974, 501201475, 501201567, 501201735, 501856957, 501857900, 501858422, 501858616, 501859216, 501859969, 501859472, 501857742, 501856252, 501857488, 501857330, 501856788, 501856534, 501855906, 501858038, 501855826, 502535851, 502536414, 502516054, 502517418, 502521145, 502519511, 502511299, 502511533, 502512472, 502511655, 502511946, 502512917, 502513356, 502513889, 502514089, 502514826, 504278114, 504278133, 504278348, 504278332, 504278214, 504278368, 504278230, 504278235, 504278240, 504278693, 504278666, 504278533, 504278323, 504278374, 504278253, 504278246, 504278385, 504278637, 504278681, 504853436, 504856968, 504857210, 504855818, 504856245, 504855495, 504853860, 504854475, 504854722, 504853222, 504853050, 504852981, 504854969, 504852878, 504855262, 504852618, 504855605, 504854157, 504854146, 504854233, 504856484, 504856580, 505538206, 505542937, 505542680, 505539971, 505540168, 505540403, 505540558, 505541083, 505541282, 505541491, 505541661, 505543435, 505542363, 505541833, 505538648, 505538998, 505542005, 505539104, 505539412, 505539647, 505541009, 505537684, 506258636, 506253987, 506255360, 506259129, 506252179, 506258208, 506260213, 506259568, 506260479, 506252458, 506252990, 506252710, 506253346, 506254749, 506255839, 506254099, 506254429, 506256265, 506257857, 506253882, 506253743, 506257278, 506251878, 506974639, 506975269, 506975645, 506976010, 506970503, 506970790, 506970946, 506971121, 506971407, 506973737, 506973949, 506974148, 506971531, 506971721, 506971959, 506972306, 506973147, 506973703, 506974350, 506976103, 508757739, 508755235, 508756922, 508757211, 508757866, 508757339, 508755566, 508755647, 508755831, 508756112, 508755033, 508754757, 508755410, 508756315, 508756635, 508756521, 508757510, 508757616, 509257945, 509264189, 509264792, 509256296, 509254753, 509255777, 509259222, 509258624, 509259846, 509260477, 509260757, 509261327, 509261667, 509262272, 509262701, 509263580, 509266838, 509265871, 509972549, 509984739, 509984872, 509975595, 509973029, 509980637, 509974599, 509973482, 509978082, 509979733, 509979103, 509981311, 509981978, 509983192, 509984382, 509983740, 509984030, 509984582, 509984878, 510700620, 510706336, 510688741, 510705319, 510706993, 510707513, 510699726, 510697354, 510694945, 510694306, 510703578, 510702668, 510693670, 510692703, 510701720, 510691869, 510690984, 510690679, 510689906, 510704038, 510708147, 511528905, 511527414, 511523982, 511523200, 511515788, 511514507, 511525062, 511516454, 511529093, 511526818, 511526274, 511526366, 511526042, 511525397, 511526291, 511517626, 511520389, 511520591, 511519007, 511521915, 511526344, 511525662, 511526323, 511526237, 514336631, 514336783, 514343251, 514340191, 514340817, 514341071, 514342045, 514336381, 514344358, 514344341, 514343637, 514342964, 514342188, 514342756, 514342341, 514343985, 514343862, 514343691, 514342470, 514339868, 514827044, 514826309, 514825891, 514826549, 514823182, 514825767, 514828847, 514827970, 514828379, 514825572, 514823482, 514824862, 514824831, 514824766, 514824638, 514828994, 514829257, 514824355, 514823988, 515480625, 515481411, 515478713, 515478945, 515476773, 515476730, 515479417, 515479472, 515479698, 515480286, 515481658, 515479120, 515477430, 515477290, 515477053, 515478523, 515478482, 515478267, 515478016, 515476573, 516110224, 516108162, 516106627, 516112390, 516110559, 516112990, 516113386, 516109831, 516111070, 516112010, 516111418, 516112679, 516113833, 516109412, 516109291, 516109020, 516108761, 516109473, 516104267, 516746744, 516747436, 516748004, 516748290, 516745541, 516745818, 516744027, 516746459, 516747087, 516747223, 516747824, 516744429, 516744642, 516744452, 516744803, 516745012, 516745194, 516743939, 518928385, 518929530, 518928069, 518927283, 518929816, 518930140, 518929730, 518927875, 518927640, 518927547, 518927441, 518929226, 518929036, 518928782, 518929348, 518929742, 518930060, 519168243, 519168036, 519167120, 519167809, 519168638, 519168561, 519167695, 519167610, 519167468, 519169786, 519169305, 519169428, 519167074, 519167219, 519167329, 519169604, 519169708, 519169727, 519169751, 519829756, 519830253, 519827388, 519825469, 519827804, 519831006, 519826509, 519826821, 519827040, 519828211, 519828815, 519828541, 519828988, 519831264, 520271441, 520270649, 520268721, 520272075, 520268530, 520271624, 520271860, 520270502, 520271269, 520270410, 520271046, 520271781, 520269674, 520269697, 520268970, 520271012, 520268260, 520272222, 520267613, 520270318, 520270165, 520270118, 520269860, 520269720, 520696029, 520696542, 520696881, 520697139, 520703669, 520704032, 520700660, 520701609, 520701059, 527662684, 520695079, 520702238, 520695150, 520695639, 520703311, 520703398, 520703849, 520702805, 520702447, 520701907, 520700099, 520699822, 520699357, 520699229, 520698560, 520702065, 520700318, 520700319, 522242753, 522237849, 522242144, 522239894, 522240559, 522243017, 522238588, 521627278, 521626099, 521625641, 521628798, 521628522, 521629132, 521881230, 521628949, 521626546, 521626372, 521628320, 521628250, 521628070, 521628060, 521627801, 521627578, 521627065, 521626898, 521883442, 521883581, 521880795, 521881124, 521880339, 521884313, 521884498, 521881433, 521882868, 521881975, 521883996, 521883114, 521883251, 521884127, 521884223, 521882958, 521882459, 522241015, 522241788, 522247455, 522248298, 522248437, 522248636, 522248789, 522246304, 522247460, 522246215, 522247418, 522246615, 522606720, 522606438, 522605458, 522604584, 522604059, 522607045, 522604239, 522606129, 522606028, 522605616, 522605574, 522605207, 522605388, 522604866, 522604414, 522605882, 522606359, 522606273, 522605805, 523054238, 523048636, 523049158, 523050076, 524024520, 523054787, 523066013, 523048102, 523097359, 523051912, 523053119, 523046870, 523047278, 523047406, 523048959, 523125181, 523077628, 523051042, 524024305, 524025688, 524027112, 524026122, 524025822, 524024626, 524025226, 524025095, 524026907, 524026805, 524026028, 524025927, 524025747, 524024660, 524024747, 524024793, 524024932, 524024962, 524025481, 524025007, 524279017, 524276951, 524275925, 524282402, 524282495, 524276378, 524275728, 524281086, 524282064, 524282103, 524282329, 524280637, 524280872, 524281893, 524281973, 524281288, 524281368, 524281427, 524281861, 524281594, 524281890, 524281523, 524276781, 524276578, 524638168, 524641581, 524639578, 524639566, 524638779, 524639598, 524639574, 524637805, 524637393, 524639631, 524637087, 524636856, 524640878, 524639362, 524640815, 524637333, 524639585, 524641428, 524639564, 524639547, 524640702, 524639634, 524639301, 524638028, 524639706, 524653823, 524992214, 524992390, 524992716, 524993397, 524996156, 525370947, 525370628, 524637677, 524639622, 524639588, 524639107, 524639017, 524638641, 524638451, 524978467, 524991329, 524997221, 524997432, 525371925, 525382147, 525385920, 524993631, 524996347, 524995417, 524996941, 524997055, 524995088, 524996583, 524996788, 524994979, 524996636, 525372229, 525372499, 525375154, 525380297, 525382622, 525385007, 525383380, 525383826, 525384763, 525385620, 525385772, 525327265, 525376047, 525376583, 525379147, 525378168, 525381947, 525382376, 525384414, 525385310, 525385350, 525382374, 526470673, 526470427, 526472037, 526470263, 526469664, 526471527, 526471423, 526471332, 526471171, 526471158, 526470886, 526469397, 526470050, 526469876, 526469845, 526469625, 526469521, 526469434, 526772539, 526774240, 526774455, 526773560, 526773069, 526773782, 526773209, 526773370, 526773687, 526771553, 526771998, 526772191, 526773939, 527180656, 527185795, 527185438, 527185179, 527184641, 527186061, 527182525, 527182946, 527181718, 527181821, 527183354, 527183402, 527183884, 527183841, 527184865, 527184615, 527184126, 527589305, 527589179, 527588936, 527588822, 527588666, 527587241, 527590644, 527590744, 527590905, 527589724, 527590575, 527587960, 527587822, 527587406, 527589960, 527590070, 527590222, 527591030, 528023013, 528022849, 528023534, 528023346, 528020862, 528021293, 528021183, 528022261, 528023820, 528022035, 528021735, 528021455, 528022656, 528021360, 528022383, 528942947, 528942928, 528943531, 528943623, 528942569, 528943450, 528943349, 528943180, 528943074, 528943071, 528942687, 528942213, 528943064, 528942384, 528942051, 528942500, 529177777, 529178430, 529175315, 529178724, 529175583, 529178068, 529177155, 529177358, 529175711, 529177585, 529175940, 529176225, 529177239, 529176368, 529176494, 529176807, 529177625, 529527933, 529523083, 529524379, 529524902, 529527788, 529527435, 529526975, 529526767, 529527446, 529527637, 529523716, 529523840, 529524054, 529524240, 529527131, 529526218, 529526507, 529525682, 529526808, 529876762, 529877215, 529877272, 529877233, 529874280, 529874704, 529873909, 529875180, 529876595, 529876368, 529876960, 529876207, 529875390, 529875061, 529874979, 529875026, 529874969, 529876853, 530294636, 530293672, 530287845, 530291424, 530287604, 530293905, 530294281, 530295115, 530294517, 530288492, 530289185, 530288693, 530286234, 530290774, 530285445, 530290382, 530289934, 530292781, 530293341, 530291838, 531174922, 531173744, 531174896, 531175628, 531175702, 531174136, 531174320, 531174372, 531175174, 531174611, 531175050, 531174453, 531175142, 531175093, 531174997, 531175475, 531175515, 531175594, 531174228, 531416177, 531416508, 531417316, 531415999, 531417552, 531415231, 531416331, 531416627, 531418517, 531419603, 531419683, 531420338, 531419170, 531416895, 531415580, 531419502, 531415823, 531415440, 531417348, 531418115, 531419016, 531418777, 531418235, 531418868, 531417958, 531418233, 531417956, 531419272, 531419281, 531419348, 531762048, 531760790, 531760723, 531760581, 531762907, 531762547, 531763495, 531768326, 531768541, 531769243, 531769080, 531766258, 531765907, 531761480, 531761725, 531764346, 531761020, 531760926, 531768834, 531768626, 532118208, 531763713, 531765124, 531765486, 531764708, 531764885, 531760345, 531759517, 531764175, 531763917, 531767214, 531768046, 531768219, 531769376, 532111437, 532123198, 532122262, 532116798, 532122602, 532115986, 532115644, 532116390, 532112106, 532115219, 532112228, 532112533, 532113104, 532114794, 532113767, 532120371, 532121862, 532125399, 532124906, 532121543, 532272655, 532260146, 532261008, 532271229, 532274040, 532112796, 532120126, 532121719, 532121697, 532123807, 532121689, 532124114, 532124401, 532123555, 532123038, 532265847, 532269933, 532265219, 532266446, 532266610, 532268007, 532268889, 533546079, 533546391, 532259279, 533547164, 533547078, 533547784, 533546788, 533546714, 533547469, 533546648, 533546562, 533546521, 533547971, 533547886, 533547725, 533547627, 533547583, 533547222, 533547300, 533546958, 533546881, 533546472, 533547394, 533782933, 533784148, 533787432, 533784288, 534150097, 533783824, 533783362, 533783497, 533783652, 533783122, 533785155, 533783336, 533783970, 533784551, 533784370, 533785688, 533786736, 533786388, 534150157, 534148903, 534150179, 534150198, 534156268, 534151000, 534152582, 534152756, 534153079, 534155604, 534155853, 534155943, 534150325, 534150476, 534150694, 534150808, 534160195, 534156800, 534159491, 534158205, 534157655, 534157518, 534157022, 534157108, 534157883, 534159029, 534158508, 534159237, 534160561, 534509199, 534509115, 534508672, 534508549, 534508897, 534508701, 534503250, 534509813, 534508144, 534509438, 534508360, 534507334, 534507820, 534507939, 534506881, 534504812, 534504396, 534508243, 534507070, 534503720, 534504065, 534504313, 534505485, 534879595, 534879735, 534881592, 534881602, 534881565, 534881287, 534881108, 534879878, 534878742, 534878628, 534878206, 534881529, 534881401, 534880187, 534880990, 534880796, 534880061, 534880264, 534879432, 534880441, 534879404, 534879217, 534879154, 534879061, 534879387, 534880995, 534881345, 535826176, 535826350, 535823576, 535824548, 535826971, 535825958, 535825866, 535825757, 535825487, 535824927, 535825086, 535825199, 535825305, 535825588, 535825997, 535825389, 535826050, 535826600, 535824082, 535823657, 535823757, 535823923, 535824286, 536068994, 536067364, 536068859, 536066371, 536066181, 536068724, 536096707, 536066613, 536066692, 536067141, 536066754, 536066152, 536067876, 536067739, 536067523, 536068498, 536067210, 536067946, 536068071, 536068212, 536068313, 536068378, 536068621, 536423509, 536424168, 536423328, 536419935, 536803437, 536423956, 536424053, 536424269, 536424374, 536422312, 536423206, 536424467, 536423052, 536422913, 536422761, 536424540, 536424758, 536421401, 536420992, 536420809, 536802671, 536799995, 536802991, 536803392, 536800351, 536800465, 536800548, 536801328, 536803330, 536803252, 536803195, 536802024, 536802208, 536802294, 536802301, 536802529, 536804873, 536803123, 536802758, 537178559, 537178412, 537178365, 537176853, 537177141, 537177843, 537177943, 537178043, 537177823, 537177499, 537177767, 537177376, 537178125, 537178223, 537178281, 537179404, 538132013, 538131615, 538131609, 538131666, 538131582, 538133036, 538131656, 538131661, 538133111, 538131700, 538131677, 538131671, 538131932, 538131787, 538131594, 538131675, 538131639, 538131636, 538131630, 538131625, 538131620, 538131645, 538373556, 538374077, 538374262, 538369218, 538369567, 538369781, 538373051, 538370273, 538375921, 538376659, 538375657, 538375402, 538375812, 538372778, 538370995, 538374758, 538370366, 538375011, 538559425, 538559067, 538554183, 538553556, 538554700, 538552161, 538552625, 538552858, 538553866, 538557088, 538554589, 538556575, 538556194, 538555690, 538554812, 538952539, 538955351, 538954890, 538951187, 538955841, 538952537, 538952322, 538954556, 538953862, 538952264, 538952129, 538951599, 538952025, 538952225, 538954165, 539406043, 539406370, 539406457, 539406249, 540346731, 540345646, 540346353, 540346413, 540346580, 539026202, 539026140, 539026080, 539026001, 539025896, 539025682, 539025516, 539025451, 539025231, 539024997, 539024867, 539406620, 540345497, 540606361, 540346901, 540344929, 540347267, 540347425, 540347572, 540347622, 540347669, 540347684, 540347690, 540347929, 540348009, 540347843, 540347768, 540603549, 540603536, 540604672, 540607484, 540603369, 540603245, 540607141, 540607318, 540604197, 540604308, 540605045, 540603953, 540604117, 540603593, 540603579, 540985738, 540989513, 540989993, 540990370, 540983961, 540984487, 540985135, 540987327, 540985935, 540986129, 540986235, 540988866, 540989392, 540986661, 540986842, 540987004, 540987598, 540988198, 540988367, 540988563, 540988797, 540988837, 540989216, 541357121, 541356055, 541355939, 541356608, 541357083, 541357287, 541357774, 541357867, 541358784, 541358982, 541356343, 541356493, 541356885, 541356934, 541357206, 541357404, 541357504, 541357610, 541357979, 541358397, 541358117, 541358185, 541358568, 541738470, 541738356, 541736362, 541739172, 541739015, 541738899, 541738819, 541738130, 541738763, 541738144, 541738578, 541736611, 541738021, 541737748, 541737721, 541737588, 541736847, 542641367, 542640764, 542641081, 542639997, 542640443, 542640565, 542640472, 542640369, 542640316, 542640309, 542640225, 542640196, 542640112, 542916237, 542915901, 542915853, 542915605, 542915570, 542913866, 542915060, 542914111, 542914032, 542915669, 542915925, 542915722, 542915341, 542915348, 542915079, 542915071, 542914230, 543257886, 543258157, 543258288, 543256517, 543257729, 543257359, 543257453, 543256903, 543257190, 543256242, 543257156, 543257057, 543257005, 543256145, 543255326, 543606615, 543604301, 543604548, 543605359, 543605885, 543603998, 543603382, 543606134, 543606757, 543606884, 543607096, 543607360, 543603493, 543603298, 543603488, 543603828, 543603685, 543603390, 543602917, 543924816, 543925297, 543925215, 543923582, 543923457, 543925089, 543924920, 543925063, 543924699, 543924655, 543924269, 543924544, 543924365, 543923835, 543923719, 544687828, 544676435, 544687708, 544686720, 544688394, 544688490, 544687654, 544687571, 544687346, 544688317, 544687949, 544688087, 544688118, 544688163, 544687236, 544687022, 544686925, 544686828, 544688236, 544687612, 544687441, 544687362, 544918020, 544921283, 544917613, 544918987, 544920317, 544917440, 544917552, 544917667, 544917714, 544917789, 544917846, 544917909, 544920343, 544920600, 544920856, 544921064, 545251186, 545251205, 545251210, 545251400, 545251420, 545251213, 545251426, 545251432, 545251435, 545251447, 545251452, 545251440, 545251463, 545251460, 545251465, 545251469, 545251474, 545251479, 546756314, 546756134, 546756786, 546756451, 546756509, 546756615, 546757006, 546757543, 546758911, 546758775, 546757699, 546757897, 546758168, 546758389, 546758542, 546759652, 546759510, 546759199, 546756418, 546756387, 546757352, 547002138, 547002373, 547002659, 547003341, 547003706, 547004298, 547008674, 547005703, 547004916, 547005023, 547005091, 547005118, 547005199, 547005458, 547319326, 547319112, 547319041, 547674813, 547003171, 547004072, 547003982, 547007131, 547004948, 547007443, 547007574, 547007781, 547007822, 547320830, 547319674, 547320629, 547322493, 547321552, 547322661, 547323040, 547324720, 547326125, 547675354, 547320367, 547327555, 547322366, 547321812, 547321712, 547676829, 547674925, 547675151, 547676153, 547676388, 547676266, 547677540, 547677612, 547676497, 547674637, 547677878, 547675463, 547675881, 547675972, 547676601, 547676754, 547677127, 547678187, 547678034, 547678390, 547676116, 547981978, 547981925, 547981919, 547981970, 547981989, 547982021, 547982024, 547982029, 547982036, 547981963, 547981966, 547981986, 547981996, 547982004, 547982011, 547982015, 548804529, 548805352, 548805159, 548804583, 548804918, 548805254, 548806102, 548806227, 548806641, 548806734, 548804874, 548804948, 548805066, 548805765, 548805851, 548806038, 548806573, 548806356, 548806465, 549055151, 549055354, 549056576, 549051567, 549053470, 549053546, 549053610, 549054492, 549054328, 549051448, 549051702, 549051792, 549053332, 549054319, 549054181, 549054069, 549053964, 549053943, 549055546, 549055621, 549055731, 549055800, 549055871, 549056070, 549056181, 549056340, 549056365, 549056509, 549054403, 549382657, 549382133, 549381883, 549383465, 549385472, 549385574, 549385638, 549718260, 549382065, 549382016, 549382010, 549385106, 549385209, 549383801, 549383689, 549383955, 549384101, 549384209, 549384991, 549385330, 549715592, 549715675, 549715868, 549716289, 549717150, 549717222, 549717271, 549717350, 549717826, 549718790, 549718694, 549716127, 549715938, 549716029, 549716188, 549716657, 549716968, 549717090, 549718400, 549717930, 549718002, 549718290, 549717418, 549717510, 549717650, 549717706, 549717790, 549718484, 549718576, 550062362, 550062266, 550062108, 550063314, 550063471, 550063943, 550062543, 550063723, 550062451, 550062616, 550062254, 550062644, 550062656, 550062908, 550063031, 550063111, 550063193, 550063557, 550063603, 550899535, 550899699, 550899233, 550899262, 550899410, 550899467, 550899798, 550900229, 550900171, 550900285, 550899875, 550899118, 550899148, 551252017, 551251685, 551252875, 551253082, 551253018, 551254112, 551254223, 551252359, 551703744, 551705745, 551250130, 551252571, 551252660, 551253209, 551253393, 551253495, 551253601, 551253697, 551253759, 551253854, 551254015, 551253938, 551705944, 551706098, 551704765, 552177190, 552179762, 551705061, 551705135, 551704225, 551704024, 551704422, 551705429, 551705582, 551706359, 552179758, 552177916, 552181518, 552178311, 552177979, 552181835, 552179474, 552179897, 552180729, 552180905, 552181137, 552181282, 552586061, 552586556, 552586679, 552586726, 552585223, 552586219, 552586260, 552585423, 552585357, 552585310, 552585651, 552585994, 552586127, 552586179, 552586321, 552586359, 552586402, 553459093, 553458985, 553457464, 553457641, 553457723, 553458689, 553459170, 553457157, 553456585, 553458277, 553457316, 553457588, 553458811, 553458749, 553458506, 553458380, 553458050, 553782857, 553783294, 553784014, 553784195, 553784446, 553785222, 553781538, 553782059, 553782078, 553780693, 553783498, 553783654, 553783791, 553785554, 553785955, 553786323, 553786137, 554188335, 554187946, 554186543, 554186781, 554189313, 554189421, 554187708, 554189689, 554189958, 554188475, 554188834, 554186969, 554187123, 554187349, 554189008, 554188141, 554190132, 554595166, 554595515, 554596032, 554596135, 554596356, 554594767, 554594628, 554594990, 554594010, 554594250, 554592986, 554593051, 554592890, 554593074, 554593303, 554595723, 554593377, 554593432, 554593556, 554594326, 554983368, 554984279, 554983476, 554984059, 554984137, 554984813, 554983052, 554984502, 554984657, 554983060, 554985136, 554983301, 554983077, 554984820, 554984981, 554983594, 555676000, 555673589, 555674378, 555674729, 555699574, 555674583, 555673948, 555674071, 555674134, 555674195, 555674231, 555673344, 555673435, 555673492, 555673620, 555673687, 555673739, 555673791, 555673827, 555673875, 555920720, 555920903, 555920474, 555920746, 555921277, 555920961, 555921060, 555921097, 555921226, 556242882, 556243557, 556241501, 556241703, 556242044, 556242949, 556243088, 556243410, 556579629, 556579661, 556579182, 556579105, 556579238, 556579352, 556579441, 556579552, 556578731, 556578847, 556578951, 556579021, 556579849, 556580933, 556934109, 556934206, 556934345, 556934535, 556932858, 556933526, 556933653, 556933699, 556933788, 556933889, 556934270, 556934402, 556933142, 556933383, 556933426, 557765878, 557765885, 557766663, 557765509, 557765551, 557765556, 557765725, 557766458, 557766603, 557765375, 557765463, 557765796, 557766007, 557766057, 557766221, 557766348, 557766139, 557766389, 557766498, 557766568, 557766790, 558027748, 558027906, 558028012, 558028187, 558028665, 558028687, 558028747, 558029155, 558029501, 558027710, 558027678, 558030290, 558030356, 558027726, 558027732, 558027740, 558028688, 558028872, 558029000, 558027719, 558356580, 558356615, 558356622, 558356604, 558356587, 558356619, 558356593, 558356621, 558356617, 558725444, 558725641, 558726972, 558728240, 558725167, 558726038, 558727772, 558727524, 558727124, 558726547, 558727873, 558726372, 558728087, 558726585, 558726841, 558728026, 558728586, 559083775, 559083582, 559084940, 559085323, 559085109, 559084754, 559083986, 559083257, 559083047, 559084518, 559082923, 559084199, 559082106, 559082058, 559084293, 559918212, 559919782, 559920007, 559920930, 559921045, 559918769, 559919438, 559921251, 559919151, 559921296, 559921379, 559920776, 559921433, 559921471, 560165170, 560166817, 560165677, 560165793, 560165985, 560165284, 560166685, 560166049, 560166215, 560166318, 560164964, 560167541, 560167695, 560167844, 560167214, 560166530, 560167290, 560167394, 560167072, 560167002, 560166940, 560167717, 560502698, 560502724, 560502635, 560502591, 560502525, 560502420, 560501335, 560503353, 560504466, 560500582, 560504261, 560499731, 560500873, 560501748, 560504238, 560504047, 560504013, 560503919, 560502748, 560502825, 560501116, 560502300, 560501521, 560502135, 560849174, 560849276, 560847814, 560847928, 560849366, 560849445, 560849079, 560848743, 560848601, 560848478, 560847197, 560847437, 560848885, 560848394, 560848297, 560848235, 560847719, 560848001, 560848202, 561178805, 561180852, 561179281, 561178632, 561183230, 561191719, 561192287, 561192559, 561192169, 561191054, 561191138, 561183048, 561182674, 561181192, 561176978, 561191517, 561191390, 561191679, 561191276, 561188675, 561188816, 561189114, 561190168, 561190533, 561190750, 561190899, 561190942, 561188928, 562041514, 562040540, 562042094, 562041214, 562041202, 562041139, 562042005, 562042052, 562041805, 562041441, 562041379, 562041601, 562041721, 562041764, 562041885, 562041947, 562040967, 562040748, 562040352, 562042621, 562272945, 562273660, 562273773, 562273898, 562274332, 562274229, 562273513, 562273342, 562273551, 562273612, 562274008, 562274219, 562274256, 562274173, 562274422, 562274098, 562274246, 562603506, 562605190, 562605601, 562605779, 562606088, 562604167, 562605033, 562606481, 562604573, 562604784, 562604415, 562606590, 562605440, 562606198, 562606259, 562606315, 562927643, 562921367, 562922259, 562922744, 562924199, 562925115, 562929388, 562926725, 562927265, 562928239, 562929253, 562927830, 562928725, 563269932, 562929406, 562926703, 562929411, 562928739, 562926714, 562920077, 562925398, 562928554, 562926563, 562926282, 562929397, 562923916, 562923028, 562917394, 563263666, 563261774, 563262693, 563263052, 563263906, 563263313, 563263462, 563263172, 563262904, 563264125, 563264216, 563264259, 563263703, 563265636, 563263594, 563262347, 564083782, 564083508, 564082411, 564083345, 564082849, 563263801, 563263843, 563263873, 563265654, 563265772, 563264903, 563265224, 563265346, 563265427, 564085555, 564084909, 564084969, 564085021, 564085593, 564084695, 564085405, 564085433, 564085493, 564085497, 564084768, 564085229, 564084833, 564085232, 564085162, 564085310, 564085342, 564299738, 564299513, 564300155, 564300046, 564299296, 564299161, 564300514, 564300322, 564300317, 564299978, 564299840, 564299617, 564299680, 564299486, 564298907, 564607786, 564609148, 564609020, 564608811, 564612049, 564607343, 564611974, 564612117, 564609694, 564609436, 564610087, 564608540, 564610192, 564610309, 564611571, 564611708, 564608392, 564608301, 564611897, 564611356, 564927236, 564925082, 564926099, 564926282, 564927781, 564927772, 564927570, 564927512, 564927974, 564927802, 564927814, 564926595, 564926764, 564927432, 564926989, 564925697, 564925774, 564926486, 564925843, 564925999, 565242297, 565242191, 565241211, 565241359, 565241567, 565241768, 565241906, 565242081, 565242376, 565242629, 565242781, 566029286, 566027589, 566030716, 566024096, 566023783, 566027968, 566027447, 566026863, 566031190, 566028630, 566028907, 566029639, 566029812, 566023194, 566030038, 566024466, 566025119, 566030247, 566030373, 566030969, 566026133, 566025415, 566030450, 566246338, 566246288, 566246485, 566246656, 566246774, 566246899, 566247935, 566248119, 566248271, 566246169, 566247480, 566247772, 566248160, 566565878, 566566030, 566566127, 566567232, 566566930, 566566440, 566566392, 566566338, 566566229, 566567428, 566902923, 566565560, 566565670, 566566618, 566566700, 566567167, 566566793, 566567037, 566903135, 566899585, 566902610, 566902121, 566901112, 566901373, 566901512, 566901774, 566902433, 566903092, 566900369, 566900989, 566900656, 566900829, 566900864, 567256681, 567258398, 567259157, 567258642, 567257056, 567256162, 567258968, 567258795, 567256619, 567256604, 567257883, 567257275, 567257192, 567257134, 567255866, 567256658, 568112631, 568110869, 568110657, 568112294, 568112627, 568112460, 568112498, 568112035, 568111468, 568111966, 568112144, 568110579, 568112546, 568112614, 568337866, 568336821, 568336677, 568337830, 568338114, 568336044, 568336945, 568336409, 568336459, 568338320, 568337149, 568337318, 568337495, 568337655, 568337883, 568337969, 568338170, 568338450, 568648784, 568649023, 568649413, 568646194, 568646574, 568646506, 568646583, 568648355, 568645809, 568646261, 568646734, 568646943, 568647016, 568647397, 568648029, 568648561, 568648222, 568978629, 568978896, 568978463, 568978321, 568979249, 568980690, 568980390, 568980162, 568984380, 568984452, 568984682, 568977787, 568979135, 568978813, 568979142, 568980080, 568979138, 568983506, 568984555, 568983713, 568983840, 568984230, 568984334, 568984651, 568984191, 569283151, 569283723, 569283830, 569284085, 569284224, 569284588, 569286149, 569286174, 569283912, 569284074, 569285091, 569284737, 569284318, 569284439, 569286015, 569285913, 569286022, 569285796, 569285784, 570067056, 570067480, 570066719, 570066835, 570066746, 570066881, 570067691, 570066623, 570066962, 570067247, 570067145, 570067282, 570067300, 570067347, 570067677, 570067582, 570067172, 570067388, 570067402, 570283293, 570283187, 570282853, 570282660, 570282125, 570283107, 570282832, 570282996, 570282550, 570282164, 570282131, 570283225, 570281806, 570283095, 570283390, 570856135, 570852619, 570852866, 570855964, 570853113, 570855026, 570854562, 570854798, 570852448, 570855196, 570854386, 570855740, 570855914, 570855642, 570855508, 570855199, 570852901, 571174196, 571175180, 571174863, 571174505, 571173141, 571176501, 571176397, 571176067, 571175891, 571175661, 571175380, 571176299, 571174175, 571174145, 571950647, 571949886, 571947997, 571948318, 571948498, 571948975, 571951382, 571950311, 571951471, 571950142, 571949652, 571949783, 571949525, 571949634, 571951060, 571951200, 571951549, 571951304, 571949969, 571949233, 571949369, 571948846, 572157179, 572159134, 572159000, 572158783, 572157304, 572157964, 572159408, 572159410, 572159137, 572160314, 572159478, 572160204, 572159135, 572157259, 572448697, 572450332, 572450596, 572450356, 572450437, 572450623, 576413984, 576413840, 576413571, 576413378, 576413285, 576413150, 576413067, 576412985, 576412866, 576412796, 576412746, 576412656, 576412516, 576102925, 576102842, 576102509, 576102785, 576106155, 576105351, 576104981, 576104075, 576102216, 576102356, 576103216, 576106924, 576103013, 576102441, 575887764, 575887676, 575887863, 575887603, 575887301, 575887548, 575887262, 575887148, 575887063, 575887000, 575886926, 575887476, 575887449, 575887379, 575886898, 575886837, 575886794, 575081073, 575081160, 575081244, 575082017, 575082124, 575082363, 575083003, 575081535, 575081634, 575082988, 575082942, 575082834, 575082817, 575082667, 575082507, 574750361, 574752386, 574750538, 574751348, 574751966, 574752131, 574752820, 574752705, 574750320, 574752635, 574750244, 574752532, 574752271, 574402428, 574402449, 574402445, 574402467, 574402462, 574402424, 574402470, 574402396, 574402417, 574402411, 574402413, 574402403, 574402442, 574402440, 574402409, 574402431, 574033856, 574034103, 574034119, 574032683, 574033613, 574032849, 574032457, 574033531, 574032433, 574032300, 574032183, 574033397, 574032044, 574032043, 574032010, 574032006, 574032938, 574031961, 574033072, 574033319, 574033326, 573787907, 573788065, 573788226, 573788290, 573788771, 573789055, 573789299, 573789355, 573788809, 573788849, 573788922, 573788355, 573788430, 573788525, 573788993, 573788686, 573788734, 573007541, 573007715, 573007364, 573007096, 573007369, 573007283, 573007174, 576414873, 576414231, 573006723, 572744931, 572745118, 572744751, 572744393, 572744078, 572744392, 572744318, 572743925, 572740425, 572742307, 572450178, 572449962, 572450208, 572450066, 572449902, 572449754, 572449195, 572448959, 572449122, 151162835, 556261903 ]


content = content[~content['bodhi_id'].isin(content_non_en_list)].reset_index(drop=True)


# 2- De-Contraction


# 3- Removing URL's
content['content_full'] = content['content_full'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


# 4- Converting to Lower & Clear the Oppostrophy 
content['content_full'] = content['content_full'].str.lower().apply(lambda x: str(x).replace(u'\n', u' ')).apply(lambda x: str(x).replace(u"’", u'')).apply(lambda x: str(x).replace(u'“', u'')).apply(lambda x: str(x).replace(u'”', u''))


# 5- Remove punctuations
content['content_full'] = content['content_full'].str.replace('[{}]'.format(string.punctuation), '')


# 6- Remove HTML Tags
content['content_full'].apply(lambda x: re.sub('<[^<]+?>', '', x))


content



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: FutureWarning: The default value of regex will change from True to False in a future version.


,bodhi_id,published_at,primary_language,headline,full_content,media_subtype,provider_name,content_full
0,243902440,2020-01-04,en,Family business - the backbone of the economy,Ebrahim Baeshen: Family companies represent mo...,website,None,family business the backbone of the economy e...
1,244253478,2020-01-04,en,Family business - the backbone of the economy,Ebrahim Baeshen: Family companies represent mo...,newspaper,None,family business the backbone of the economy e...
2,243902505,2020-01-05,en,Responsible business: is this time different?,"This has been the year of profit with purpose,...",newspaper,None,responsible business is this time different th...
3,244749725,2020-01-06,en,Effat student wins EY corporate finance award,"Yara Bugis, a Saudi female student from Effat ...",website,None,effat student wins ey corporate finance award ...
4,244246407,2020-01-06,en,Yara Bugis Wins EY KSA Corporate Finance Woman...,"Yara Bugis, a Saudi female student from Effat ...",website,None,yara bugis wins ey ksa corporate finance woman...
...,...,...,...,...,...,...,...,...
7078,233227761,2019-12-06,en,Aramco shares to start trading Dec 11: Tadawul...,RIYADH — Saudi Aramco shares will start tradin...,website,None,aramco shares to start trading dec 11 tadawul ...
7079,231380652,2019-12-03,en,Deloitte Middle East Point Of View: Disrupt Th...,The oil and gas industry in the Middle East re...,website,None,deloitte middle east point of view disrupt the...
7080,230634371,2019-12-02,en,Global PwC study ranks Bahrain as the easiest ...,Bahrain has been deemed the easiest place on e...,website,None,global pwc study ranks bahrain as the easiest ...
7081,230633415,2019-12-02,en,GCC countries set their sights on knowledge ec...,DUBAI: The member states of the Gulf Cooperati...,website,None,gcc countries set their sights on knowledge ec...


In [8]:
# working with N-Grams, Tri-Grams


def N_Grams(text,n):
    tokens = re.split("\\s+", text)
    ngrams = []

    for i in range(len(tokens)-n+1):
        temp = [tokens[j] for j in range(i,i+n)]
        ngrams.append(" ".join(temp))

    return ngrams

text = content['content_full']

N_Grams(text[0],3)





['family business the',
 'business the backbone',
 'the backbone of',
 'backbone of the',
 'of the economy',
 'the economy ebrahim',
 'economy ebrahim baeshen',
 'ebrahim baeshen family',
 'baeshen family companies',
 'family companies represent',
 'companies represent more',
 'represent more than',
 'more than 90',
 'than 90 percent',
 '90 percent of',
 'percent of the',
 'of the total',
 'the total number',
 'total number of',
 'number of saudi',
 'of saudi companies',
 'saudi companies and',
 'companies and contribute',
 'and contribute about',
 'contribute about 12',
 'about 12 percent',
 '12 percent of',
 'percent of gdp',
 'of gdp for',
 'gdp for the',
 'for the kingdom',
 'the kingdom with',
 'kingdom with more',
 'with more than',
 'more than 40',
 'than 40 percent',
 '40 percent of',
 'percent of the',
 'of the kingdoms',
 'the kingdoms nonoil',
 'kingdoms nonoil gdp',
 'nonoil gdp ']

In [34]:
ner_model = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/Dataset/NER_data_load.xlsx")

ner_model = ner_model[ner_model['lables'].isin(['ORG','EVENT','FAC','GPE','LOC','MONEY','PERSON','PRODUCT'])].reset_index(drop=True)

ner_model['Totalwords'] = ner_model['Entitites'].str.split().str.len()

print("Mean", )

# ner_model = ner_model['Entitites'].tolist()

# ner_model = list(dict.fromkeys(ner_model))




# 107107 -- total count
# 28518 -- after remove duplicates

2.018532869000159

In [31]:
print(len(ner_model))

cv = CountVectorizer(stop_words='english',ngram_range=(1,3) ,strip_accents=None, lowercase=True, max_df=0.60, min_df=5, max_features=None, vocabulary=None)
cv.fit(ner_model)
vect_cv  = cv.fit_transform(content['content_full'][:1000])


vect_cv


print(cv.get_feature_names())

print(len(cv.vocabulary_))


# print({k: v for k, v in sorted(dtm.vocabulary_.items(), key=lambda item: item[1])})


# cv.get_feature_names()
# cv_df = pd.DataFrame(vect_cv.toarray(), columns=cv.get_feature_names())
# cv_df.head()



# ['23000km²', 'alzubi评论道', 'andré', 'angélique', 'antónio', 'appétit', 'attaché', 'attachés', 'aéro', 'bagó', 'benoîtetienne', 'bioegénesis', 'biogénesis', 'björn', 'blériots', 'bohème', 'botín', 'bágo', 'bödeker', 'börje', 'börse', 'café', 'cafés', 'camagüey', 'camagüeyrequejo', 'capuçon', 'cen', 'chargé', 'chaussées', 'cinépolis', 'cissé', 'communiqué', 'corporación', 'cov', 'cs²ai', 'côte', 'desormière', 'décor', 'défense', 'dêtre', 'dübel', 'emiroğlu', 'entrée', 'français', 'française', 'frontières', 'frédéric', 'frérot', 'futtaim公司将为每个承诺植一棵树', 'futtaim的可持续发展之旅差不多始于15年前', 'futtaim的目标', 'fédération', 'gamiﬁed', 'grabarkitarović', 'gutiérrez', 'générale', 'géologiques', 'gérard', 'güner', 'güneş', 'holding首席可持续发展官ibrahim', 'häagendazs', 'häkkinen', 'hélène', 'hôtelière', 'internationål', 'interprotección', 'intra', 'isc²', 'jetñilkijiner', 'jørgen', 'kjær', 'kıvanç', 'loréal', 'lévy', 'maiquetía', 'marcandré', 'maëlle', 'minières', 'montréal', 'musée', 'm²', 'médecins', 'müller', 'nestlé', 'nürburgring', 'oudéa', 'parlia', 'pará', 'protégé', 'protégées', 'renée', 'rtx', 'rémi', 'saïd', 'saôneetloire', 'schaart', 'société', 'systèmes', 'são', 'sébastien', 'sélim', 'taghzout', 'takāfulnd', 'ter', 'thunderbolt', 'timecritical', 'tüv', 'ultem', 'vendôme', 'visàvis', 'västerås', 'walsh评论道', 'zaïdou', 'álvaro', 'école', 'الضرائب', 'ḥimā', '一年中最美好的时光快速临近', '为每一天', '为每个人', '今年', '今年购物中心最大的圣诞奇观是一棵非常吸睛的八米高圣诞树安装活动', '从中午12点到午夜', '从现在起一直到12月25日', '作为可持续发展的领导者', '作为社区', '假日季是感恩时光', '创造更美好的时光', '可回收利用率增加了32', '圣诞季真的是一种归家的欢庆', '圣诞精神和团聚来纪念', '在这段时间', '它向阿联酋购物中心的游客和全世界发出了强有力的信号', '客户还可以购买以节日为主题的饰品和礼物送给心仪的人', '家人和社区被拉得更近了', '展示的主题围绕气候变化和可持续发展', '并伴有宏大的体验活动', '并帮助世界履行其可持续发展使命', '慷慨', '我们majid', '我们不断寻求各种方法来减少自身对环境的影响', '我们取得了令人难以置信的进展', '我们希望发出季节性欢呼', '我们所有人都有责任支持和保护环境', '我们正在提高对这一重要问题的认识', '我们生产了3百万千瓦时的可再生能源', '提供丰富的圣诞美食和可口的饮料', '朋友', '朝着到2040年实现净正面影响的目标迈进', '欢迎游客通过独特的圣诞誓言对更可持续的生活作出承诺', '由独特的供应商组成', '碳排放减少了4', '突出强调社区如何能协力减少其对环境的影响', '美通社', '让他们沉浸其中', '该用欢乐', '购物中心营销和传播高级总监michelle', '购物者和购物中心游客打造一个欢乐和冒险的世界', '这也折射了我们majid', '这棵碳中和led灯圣诞树将进行富有冲击力的展示', '通过将可持续性纳入今年圣诞庆祝活动的核心', '阿联酋准备好了攀登全球可持续发展浪潮的巅峰', '阿联酋购物中心启动了独特的坚定面向未来的圣诞安装活动', '阿联酋购物中心启动圣诞季向未来发出强有力信号', '阿联酋购物中心将为家庭', '阿联酋购物中心的假日活动安排牢固地扎根于未来', '阿联酋购物中心的圣诞季还将包括一个圣诞集市', '阿联酋购物中心的用水下降了7', '阿联酋迪拜2021年12月13日', 'ﬁscal']



28518
['08', '08 higher', '08 higher 2019', '10', '100', '1014', '1014 million', '1014 million saudi', '101bln', '101bln end', '101bln end h1', '1020', '1020 personal', '1020 personal loans', '11', '12', '12 months', '13', '14', '15', '16', '19', '19 king', '19 king abdulaziz', '20', '20 ceos', '20 ceos kingdom', '20 engagement', '20 engagement group', '20 personal', '20 personal loans', '20 w20', '20 w20 official', '20 w20 presided', '2008', '2015', '2015 turkish', '2015 turkish presidency', '2016', '2018', '2018 ranked', '2018 ranked sixth', '2019', '2019 estimates', '2019 estimates supported', '2019 excise', '2019 excise tax', '2019 nonetheless', '2019 nonetheless saudi', '2019 performance', '2019 performance tadawullisted', '2019 total', '2019 total written', '2020', '2020 according', '2020 ad', '2020 ad maintaining', '2020 compared', '2020 compared 34119', '2020 gross', '2020 gross written', '2020 hr', '2020 hr pulse', '2020 kpmg', '2020 report', '2020 report based', '2020 results

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
# filtered_sentences = [w for w in cv2.get_feature_names() if not w in cv.get_feature_names()]

# print(filtered_sentences)

# ['23000km2', 'alvaro', 'appetit', 'bago', 'benoitetienne', 'bioegenesis', 'biåogenesis', 'bjorn', 'bleriots', 'bodeker', 'boheme', 'borje', 'borse', 'camaguey', 'camagueyrequejo', 'capucon', 'cisse', 'coreTM', 'corporacion', 'cs2ai', 'desormiere', 'detre', 'emiroglu', 'entree', 'frederic', 'frerot', 'frontieres', 'funtasiaTM', 'geologiques', 'gerard', 'grabarkitarovic', 'guner', 'gunes', 'gutierrez', 'haagendazs', 'helene', 'hoteliere', 'interproteccion', 'isc2', 'jetnilkijiner', 'kıvanc', 'maiquetia', 'marcandre', 'mattersTM', 'medecins', 'minieres', 'montreal', 'musee', 'nurburgring', 'oudea', 'protege', 'protegees', 'renee', 'rtxTM', 'sao', 'saoneetloire', 'sebastien', 'selim', 'takafulnd', 'thunderboltTM', 'ultemTM', 'vasteras', 'vendome', 'waveTM', 'zaidou', 'الضرايب']


arr_cv = []

res = [sub for sub in arr_cv if all((ele in sub) for ele in ['written'])]

print(res)



NameError: ignored

In [ ]:
# filtered_sentencess = [w for w in cv1.get_feature_names() if not w in cv.get_feature_names()]

# print(filtered_sentencess)

# ['1015yearolds', '18002000', '20152016', '20172018', '2018kpmg', '20212045', '2030a', '2030crown', '2030still', '23000km2', '26c', '79f', 'accelerateor', 'alvaro', 'appetit', 'areawill', 'assetshave', 'authorityhas', 'bago', 'benoitetienne', 'beyondcontinue', 'bioegenesis', 'biogenesis', 'bjorn', 'bleriots', 'blueprintthe', 'bodeker', 'boheme', 'borje', 'borse', 'businessesfor', 'camaguey', 'camagueyrequejo', 'capucon', 'cisse', 'coreTM', 'corporacion', 'countriesthat', 'cs2ai', 'customerslocal', 'desormiere', 'detre', 'e300', 'emiroglu', 'entree', 'frederic', 'frerot', 'frontieres', 'funtasiaTM', 'geologiques', 'gerard', 'grabarkitarovic', 'grouphave', 'guner', 'gunes', 'gutierrez', 'haagendazs', 'helene', 'hinderthe', 'hoteliere', 'increasesmust', 'infrastructureparticularly', 'initiativesdue', 'institutionsfirst', 'interproteccion', 'isc2', 'jetnilkijiner', 'jobof', 'jrgen', 'kjr', 'kvanc', 'maiquetia', 'marcandre', 'mattersTM', 'mdksa', 'medecins', 'minieres', 'montreal', 'musee', 'nontimecritical', 'nurburgring', 'operatorsboth', 'oudea', 'pledgessome', 'protege', 'protegees', 'reductionnot', 'regulatorsincluding', 'renee', 'rtxTM', 'sao', 'saoneetloire', 'schaarttaghzout', 'sebastien', 'selim', 'sumouone', 'takafulnd', 'thunderboltTM', 'uae7', 'ultemTM', 'vasteras', 'vendome', 'waryemphasises', 'waveTM', 'yorkfront', 'zaidou'] 

In [ ]:
ner_model = ['KPMG', 'Financial Times', 'McKinsey’s', 'JPMorgan Chase', 'the EY KSA Corporate Finance Woman of the Year', 'Effat University', 'Yara', 'the EY KSA Corporate Finance Woman of the Year', 'Effat University', 'King Saud University', 'AlFaisal University', 'TAS', 'Transaction Advisory Services', 'Yara', 'the ‘EY Corporate Finance Woman', 'King Abdulaziz University', 'King Saud university', 'the Corporate Finance Woman']
ner_model = list(set(ner_model))
print(len(ner_model))


tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,3) ,strip_accents=None, lowercase=True, max_df=0.50, min_df=2, max_features=None, vocabulary=None)
tfidf.fit(ner_model)
vect  = tfidf.fit_transform(content['content_full'][:1000])


vect




tfidf.get_feature_names()
tfidf_df = pd.DataFrame(vect.toarray(), columns=tfidf.get_feature_names())
tfidf_df.head()



In [ ]:

stop_words = set(stopwords.words("english"))
words = word_tokenize(content['content_full'][2])

print(type(words))
print(" ")
print(content['content_full'][2])
print(" ")

filtered_sentence = [w for w in words if not w in stop_words]
print(len(filtered_sentence))
print(filtered_sentence)



In [ ]:
# bodh_id_arr = []

# for fc in range(0,len(content)):
#     print(fc)
#     if len(re.findall(r'[\u4E00-\u9FFF]+', content['full_content'][fc])) > 0:
#         bodh_id_arr.append(content['bodhi_id'][fc])
#     time.sleep(2)

# print(bodh_id_arr)
# 556261903

# content[content['bodhi_id'] == 556261903]['content_full']

content['content_full'][5106]



In [ ]:
import nltk
nltk.download('wordnet')

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

df = pd.DataFrame(['this was cheesy', 'she likes these books', 'wow this is great'], columns=['text'])
df['text_lemmatized'] = df.text.apply(lemmatize_text)

df['text_lemmatized']



In [ ]:
content.to_pickle("/content/gdrive/My Drive/Colab Notebooks/Dataset/kpmg_full_text_export_2018-2022_filter.pkl")